In [2]:
# Import necessary libraries
import os
import glob
import pandas as pd
import numpy as np
import requests
from io import BytesIO
from PIL import Image
import matplotlib
matplotlib.use('Agg')  # Use a non-interactive backend suitable for script running
import matplotlib.pyplot as plt
from tqdm import tqdm  # For progress bars
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import torch.nn as nn

import faiss

# Step 0: Define Helper Functions
print("Step 0: Defining helper functions...")

def download_images(final_df, save_dir='../../scratch/mexas.v/art_images/*.jpg'):
    """
    Downloads images from URLs specified in the final_df DataFrame.

    Parameters:
    - final_df (pd.DataFrame): DataFrame containing 'objectid' and 'image_urls' columns.
    - save_dir (str): Directory where images will be saved.

    Returns:
    - None
    """
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        print(f"Created directory: {save_dir}")
    else:
        print(f"Directory already exists: {save_dir}")

    print("Starting image downloads...")
    for idx, row in tqdm(final_df.iterrows(), total=final_df.shape[0], desc="Downloading images"):
        objectid = str(row['objectid'])
        image_urls = row['image_urls']
        for i, image_url in enumerate(image_urls):
            image_filename = f"{objectid}_{i}.jpg"
            image_path = os.path.join(save_dir, image_filename)

            # Skip downloading if the file already exists
            if os.path.exists(image_path):
                continue

            try:
                response = requests.get(image_url, timeout=10)
                response.raise_for_status()  # Check if the request was successful
                with open(image_path, 'wb') as f:
                    f.write(response.content)
            except requests.exceptions.RequestException as e:
                print(f"Failed to download {image_filename} from {image_url}: {e}")

def show_results(image_path_or_url, results):
    """
    Displays the query image alongside its top matching results.

    Parameters:
    - image_path_or_url (str): Path or URL of the query image.
    - results (pd.DataFrame): DataFrame containing the top matching artworks.

    Returns:
    - None
    """
    print("\nDisplaying results visually...")
    try:
        if image_path_or_url.startswith('http'):
            response = requests.get(image_path_or_url, timeout=10)
            query_img = Image.open(BytesIO(response.content)).convert('RGB')
        else:
            query_img = Image.open(image_path_or_url).convert('RGB')
    except Exception as e:
        print(f"Failed to load query image: {e}")
        return

    num_results = min(len(results), 5)
    plt.figure(figsize=(5 * (num_results + 1), 5))

    # Display Query Image
    plt.subplot(1, num_results + 1, 1)
    plt.imshow(query_img)
    plt.title('Query Image')
    plt.axis('off')

    # Display Matching Images
    for i in range(num_results):
        img_path = results.iloc[i]['image_path']
        try:
            img = Image.open(img_path).convert('RGB')
            plt.subplot(1, num_results + 1, i + 2)
            plt.imshow(img)
            plt.title(f"Match {i+1}")
            plt.axis('off')
            print(f"Loaded Match {i+1}: {img_path}")
        except Exception as e:
            print(f"Failed to load image {img_path}: {e}")

    output_dir = 'results'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    output_path = os.path.join(output_dir, 'query_results.png')
    plt.savefig(output_path)
    plt.close()
    print(f"Results displayed and saved to '{output_path}'.")

def process_input_image(image_path_or_url, transform, device):
    """
    Processes the input image by loading, transforming, and sending it to the device.

    Parameters:
    - image_path_or_url (str): Path or URL of the image.
    - transform (torchvision.transforms.Compose): Transformations to apply.
    - device (torch.device): Device to send the image tensor.

    Returns:
    - torch.Tensor: Processed image tensor.
    """
    print(f"  Processing input image: {image_path_or_url}")
    try:
        if image_path_or_url.startswith('http'):
            response = requests.get(image_path_or_url, timeout=10)
            img = Image.open(BytesIO(response.content)).convert('RGB')
        else:
            img = Image.open(image_path_or_url).convert('RGB')
    except Exception as e:
        print(f"  Failed to load image {image_path_or_url}: {e}")
        # Return a black image in case of error
        img = Image.new('RGB', (224, 224), (0, 0, 0))
    img = transform(img).unsqueeze(0).to(device)
    print("  Image processed and transformed.")
    return img

def get_image_features(model, img_tensor):
    """
    Extracts and normalizes features from the image tensor.

    Parameters:
    - model (torch.nn.Module): The feature extraction model.
    - img_tensor (torch.Tensor): Image tensor.

    Returns:
    - np.ndarray: Normalized feature vector.
    """
    print("  Extracting features from the query image...")
    with torch.no_grad():
        features = model(img_tensor)
        features = features.view(features.size(0), -1)  # Flatten
        features = features / features.norm(dim=1, keepdim=True)
    print("  Features extracted and normalized.")
    return features.cpu().numpy().astype('float32')

def search_artwork(query_features, index, k=5):
    """
    Searches for the top k similar artworks using FAISS.

    Parameters:
    - query_features (np.ndarray): Feature vector of the query image.
    - index (faiss.Index): FAISS index.
    - k (int): Number of top matches to retrieve.

    Returns:
    - distances (np.ndarray): Similarity scores.
    - indices (np.ndarray): Indices of the top matches.
    """
    print(f"  Searching for top {k} similar artworks...")
    distances, indices = index.search(query_features, k)
    print("  Search completed.")
    return distances[0], indices[0]

def get_artwork_info(indices, image_df):
    """
    Retrieves artwork information based on the indices.

    Parameters:
    - indices (np.ndarray): Indices of the top matches.
    - image_df (pd.DataFrame): DataFrame containing image metadata.

    Returns:
    - pd.DataFrame: DataFrame of the top matching artworks.
    """
    print("  Retrieving artwork information for the top matches...")
    try:
        results = image_df.iloc[indices].reset_index(drop=True)
        print("  Artwork information retrieved.")
    except Exception as e:
        print(f"  Error retrieving artwork information: {e}")
        results = pd.DataFrame()  # Return empty DataFrame on error
    return results

def identify_artwork(image_path_or_url, model, index, image_df, transform, device, k=5):
    """
    Identifies the top k artworks similar to the query image.

    Parameters:
    - image_path_or_url (str): Path or URL of the query image.
    - model (torch.nn.Module): Feature extraction model.
    - index (faiss.Index): FAISS index.
    - image_df (pd.DataFrame): DataFrame containing image metadata.
    - transform (torchvision.transforms.Compose): Transformations to apply.
    - device (torch.device): Device to send the image tensor.
    - k (int): Number of top matches to retrieve.

    Returns:
    - results (pd.DataFrame): DataFrame of the top matching artworks.
    - distances (np.ndarray): Similarity scores.
    """
    print("\nIdentifying artwork...")
    img_tensor = process_input_image(image_path_or_url, transform, device)
    query_features = get_image_features(model, img_tensor)
    distances, indices = search_artwork(query_features, index, k)
    results = get_artwork_info(indices, image_df)
    print("Artwork identification completed.")
    return results, distances

def extract_features(model, dataloader, device):
    """
    Extracts features from images using the provided model and dataloader.

    Parameters:
    - model (torch.nn.Module): The feature extraction model.
    - dataloader (DataLoader): DataLoader for the dataset.
    - device (torch.device): Device to perform computations on.

    Returns:
    - torch.Tensor: Extracted features.
    """
    print("Starting feature extraction...")
    features = []
    with torch.no_grad():
        for batch_idx, (imgs, _) in enumerate(tqdm(dataloader, desc="Extracting features"), 1):
            imgs = imgs.to(device)
            try:
                outputs = model(imgs)
                outputs = outputs.view(outputs.size(0), -1)  # Flatten to (batch_size, feature_dim)
                outputs = outputs / outputs.norm(dim=1, keepdim=True)
                features.append(outputs.cpu())
            except Exception as e:
                print(f"Error processing batch {batch_idx}: {e}")
    features = torch.cat(features, dim=0)
    print("Feature extraction completed.")
    return features

# Step 1: Load and Prepare Datasets
print("\nStep 1: Loading and preparing datasets...")

try:
    objects_df = pd.read_csv('./datasets/objects.csv')
    constituents_df = pd.read_csv('./datasets/constituents.csv')
    images_df = pd.read_csv('./datasets/published_images.csv')
    print("Datasets loaded successfully.")
except Exception as e:
    print(f"Error loading datasets: {e}")
    raise

# Ensure relevant columns are strings and stripped
for df, key in [
    (objects_df, 'objectid'),
    (constituents_df, 'artistofngaobject'),
    (images_df, 'depictstmsobjectid')
]:
    df[key] = df[key].astype(str).str.strip()

# Aggregate artists
print("Aggregating artist information...")
artists_agg = constituents_df.groupby('artistofngaobject')['preferreddisplayname'] \
    .apply(lambda names: ', '.join(sorted(set(names)))).reset_index()
artists_agg.rename(columns={
    'artistofngaobject': 'objectid',
    'preferreddisplayname': 'artists'
}, inplace=True)

# Aggregate image URLs
print("Aggregating image URLs...")
images_agg = images_df.groupby('depictstmsobjectid')['iiifthumburl'] \
    .apply(list).reset_index()
images_agg.rename(columns={
    'depictstmsobjectid': 'objectid',
    'iiifthumburl': 'image_urls'
}, inplace=True)

# Merge datasets
print("Merging datasets into final_df...")
merged_with_artists = objects_df.merge(
    artists_agg,
    on='objectid',
    how='left'
)
final_df = merged_with_artists.merge(
    images_agg,
    on='objectid',
    how='left'
)

# Select and clean relevant columns
final_df = final_df[['objectid', 'title', 'artists', 'image_urls']]
final_df['artists'] = final_df['artists'].fillna('Unknown Artist')
final_df['image_urls'] = final_df['image_urls'].apply(lambda x: x if isinstance(x, list) else [])
final_df = final_df[final_df['image_urls'].map(len) > 0].reset_index(drop=True)

# Print summary statistics
print("\nSummary of final_df:")
print(f"Unique artworks in objects_df: {objects_df['objectid'].nunique()}")
print(f"Unique artists in constituents_df: {constituents_df['constituentid'].nunique()}")
print(f"Unique image URLs in images_df: {images_df['iiifthumburl'].nunique()}")
print(f"Unique artworks in final_df: {final_df['objectid'].nunique()}")
print(f"Columns in final_df: {final_df.columns.tolist()}")

# Step 2: Download Images (Optional)
# Uncomment the following lines if you need to download images
# print("\nStep 2: Downloading images...")
# download_images(final_df)

print("\nStep 3: Preparing image_df from downloaded images...")

# Check if 'image_df.pkl' already exists
if os.path.exists('image_df.pkl'):
    # Load image_df from the pickle file
    image_df = pd.read_pickle('image_df.pkl')
    print(f"Loaded image_df with {len(image_df)} entries from 'image_df.pkl'.")
else:
    image_paths = glob.glob('../../scratch/mexas.v/art_images/*.jpg')
    print(f"Found {len(image_paths)} images in './art_images/' directory.")
    
    # Use all images
    subset_image_paths = image_paths
    print(f"Selected {len(subset_image_paths)} images for processing.")
    
    # Prepare image_data
    image_data = []
    for path in tqdm(subset_image_paths, desc="Preparing image data"):
        filename = os.path.basename(path)
        objectid = filename.split('_')[0]
        row = final_df[final_df['objectid'] == objectid]
        if row.empty:
            # Skip images without metadata
            continue
        row = row.iloc[0]
        image_data.append({
            'image_path': path,
            'objectid': objectid,
            'artists': row['artists'],
            'title': row['title']
        })
    
    # Create image_df
    image_df = pd.DataFrame(image_data)
    print(f"Prepared image_df with {len(image_df)} entries.")
    
    # Save image_df to a pickle file for future use
    image_df.to_pickle('image_df.pkl')
    print("Saved image_df to 'image_df.pkl' for future use.")

# Step 4: Prepare Dataset for Feature Extraction
print("\nStep 4: Preparing dataset for feature extraction...")

# No need to filter the dataset since we want to include all images
print(f"Total number of images: {len(image_df)}")

# Step 5: Define Image Transformations
print("\nStep 5: Defining image transformations...")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

print("Image transformations defined.")

# Step 6: Create Custom Dataset and DataLoader
print("\nStep 6: Creating custom dataset and data loader...")

class ArtImageDataset(Dataset):
    def __init__(self, image_df, transform=None):
        self.image_df = image_df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.image_df)

    def __getitem__(self, idx):
        img_path = self.image_df.loc[idx, 'image_path']

        try:
            img = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            img = Image.new('RGB', (224, 224), (0, 0, 0))

        if self.transform:
            img = self.transform(img)

        return img, idx  # Return index to keep track

# Create dataset and DataLoader
batch_size = 32  # Adjust based on your GPU memory
dataset = ArtImageDataset(image_df, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=12)

print(f"DataLoader created with {len(dataloader)} batches.")

# Step 7: Set Up Device and Load Pre-trained Model
print("\nStep 7: Setting up the device and loading the pre-trained model...")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the pre-trained ResNet-50 model
model = models.resnet50(pretrained=True)

# Remove the final classification layer
model = nn.Sequential(*list(model.children())[:-1])
model = model.to(device)
model.eval()

print("Pre-trained model loaded and modified for feature extraction.")

# Step 8: Extract Features
print("\nStep 8: Extracting features from all images...")

features = extract_features(model, dataloader, device)
print(f"Extracted features shape: {features.shape}")

# Convert features to numpy array
features_np = features.squeeze().numpy().astype('float32')

# Save features
np.save('image_features.npy', features_np)
print("Features saved to 'image_features.npy'.")

# Step 9: Create FAISS Index and Add Features
print("\nStep 9: Creating FAISS index with extracted features...")

index = faiss.IndexFlatIP(features_np.shape[1])
index.add(features_np)
faiss.write_index(index, 'faiss_index.bin')
print("FAISS index created with extracted features and saved to 'faiss_index.bin'.")

# Step 10: Perform the Query on a Sample Image
print("\nStep 10: Performing query on a sample image...")

# Use any image from the dataset as the query
query_image_index = 80  # Adjust index as needed
if query_image_index >= len(image_df):
    query_image_index = len(image_df) - 1  # Ensure the index is within range
image_to_query = image_df.iloc[query_image_index]['image_path']
print(f"Selected image for querying: {image_to_query}")

try:
    # Use the same model and FAISS index
    results, distances = identify_artwork(
        image_to_query,
        model,
        index,
        image_df,
        transform,
        device,
        k=5
    )
except Exception as e:
    print(f"Error identifying artwork: {e}")
    results, distances = None, None

# Step 11: Display Top Matching Artworks
if results is not None and not results.empty:
    print("\nTop matching artworks:")
    for i, (idx, dist) in enumerate(zip(results.index, distances)):
        row = results.iloc[i]
        print(f"Rank {i+1}:")
        print(f"  Artwork Title: {row['title']}")
        print(f"  Artist: {row['artists']}")
        print(f"  Similarity Score: {dist:.4f}")
        print(f"  Image Path: {row['image_path']}\n")
else:
    print("No results to display due to an error in identifying the artwork.")

# Step 12: Display Results Visually
if results is not None and not results.empty:
    print("\nStep 12: Displaying results visually...")
    show_results(image_to_query, results)
else:
    print("Skipping visual display due to earlier errors.")


Step 0: Defining helper functions...

Step 1: Loading and preparing datasets...


KeyboardInterrupt: 